# Tuning

In [3]:
import numpy as np
import pysmurf

S = pysmurf.SmurfControl(cfg_file='')

ModuleNotFoundError: No module named 'pysmurf'

## Coarse Tuning

Coarse tuning is a quick search for resonator dips. We generate a known broad band noise signal out of the DAC. We then measure the response in the ADC. The cross correlation between these input and the output has notches where the resonators are. We refer to this process as full_band_resp. Since we know the phase of our input/output data, we have both amplitude and phase information. To do the coarse peak finding manually

In [ ]:
f, resp = S.full_band_resp(2)
peaks = S.find_peak(f, resp, make_plot=True, show_plot=True)

This coarse search is very good at finding the approximate location of all the resonators. However we need knowledge of the resonator locations in frequency to an accuracy of about 10 kHz. To further refines the peak search, a gradient descent minima search is run. This is run in pyrogue to speed up the process. This takes about 2 minutes for 150 resonators.